References:
- https://docs.python.org/3/reference/datamodel.html#emulating-container-types
- Fluent Python by Luciano Ramalho. Chapter 11: Interfaces: From Protocols to ABCs

From dynamic protocols to abstract base classes.

# Interfaces and Protocols in  Python Culture
- Protocols: informal interfaces that make polymorphism work
 - `interface` keyword
 - set public attributes

#### `x` and `y` are public data attributes:

    class Vector2d:
        typecode = 'd'

        def __init__(self, x, y):
            self.x = float(x)    
            self.y = float(y)

        def __iter__(self):
            return (i for i in (self.x, self.y)) 
    ...

#### `x` and `y` are public data attributes:

    class Vector2d:
        typecode = 'd'

        def __init__(self, x, y):
            self.__x = float(x)
            self.__y = float(y)

        @property
        def x(self):
            return self.__x

        @property
        def y(self):
            return self.__y

        def __iter__(self):
            return (i for i in (self.x, self.y))
    ...

#### Interfaces: the subset of an object's public methods that enable it to play a specific role in the system
Protocols are interfaces, but informal. Defined only by documentation and convention.
# Python Digs Sequences
<img src="sequence.png" width="50%">

#### Partial sequence protocol implementation with `__getitem__`

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [2]:
f = Foo()
f[1]

10

In [3]:
for i in f: print(i)

0
10
20


In [4]:
20 in f

True

In [5]:
15 in f

False

#### Implement both methods of the sequence protocol:

In [6]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [7]:
deck = FrenchDeck()
deck[:5]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='spades')]

In [8]:
from random import shuffle

In [9]:
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

# Monkey-Patching to Implement a Protocol at Runtime
## Monkey  Patching
Changing a class or module at runtime, without touching the source code.

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card    # assign the function to __setitem__ attribute

shuffle(deck)

deck[:5]

[Card(rank='K', suit='spades'),
 Card(rank='6', suit='hearts'),
 Card(rank='8', suit='clubs'),
 Card(rank='6', suit='spades'),
 Card(rank='6', suit='clubs')]

Key: `set_card` knows that thee `deck` object has an attribute named _cards, and must be a mutable sequence.


# Subclassing an ABC

In [11]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # to enable shuffling
        self._cards[position] = value

    def __delitem__(self, position):  # required by subclassing MutableSequence
        del self._cards[position]

    def insert(self, position, value):  # abstract method of MutableSequence
        self._cards.insert(position, value)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  """


<img src="mutablesequence.png" width="75%">

# ABCs in the Standard Library

<img src="collections.png" width="75%">

## The Numbers Tower
- Number
- Complex
- Real
- Rational
- Integral

# Defining and Using an ABC

<img src="tombola.png" width="50%">


In [12]:
import abc

class Tombola(abc.ABC):  # define an ABC subclass

    @abc.abstractmethod
    def load(self, iterable):  # abstrack method marked with the decorator
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # intructs the implementors to raise error if there are no items to pick
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # may include concrete methodd
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # rely only on the interface defined


    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # we don't know how concrete subclasses will store the items
            try:
                items.append(self.pick()) # emptying using pick()
            except LookupError:
                break
        self.load(items)  # put everything back
        return tuple(sorted(items))


In [13]:
class Fake(Tombola):
    def pick(self):
        return 13

In [14]:
Fake

__main__.Fake

In [15]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

## Subclassing the Tombola ABC

In [16]:
import random

In [17]:
class BingoCage(Tombola):  # explicitly extends Tombola

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # provides random bytes "suitable for crytographic use"
        self._items = []
        self.load(items)  # Delegate initial loading

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # using from SystemRandom

    def pick(self):  
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self): 
        self.pick()

In [18]:
class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # accepts any iterable

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # Change ValueError to LookupError to be consistent
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)  # otherwise, randomly select

    def loaded(self):  # override loaded to avoid calling inspect
        return bool(self._balls)

    def inspect(self):  # override inspect
        return tuple(sorted(self._balls))

# A Virtual Subclass of Tombola
<img src="tombolalist.png" width="50%">

In [19]:
from random import randrange


@Tombola.register  # register as a virtual subclass
class TomboList(list):  # extends list

    def pick(self):
        if self:  # inherits __bool__ from list and returns true if not empty
            position = randrange(len(self))
            return self.pop(position)  # passing a random item index
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # same as list.extend

    def loaded(self):
        return bool(self)  # delegates to bool

    def inspect(self):
        return tuple(sorted(self))

In [20]:
issubclass(TomboList, Tombola)

True

In [21]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [22]:
TomboList.__mro__

(__main__.TomboList, list, object)

# Tests

In [23]:
balls = list(range(3))
globe = BingoCage(balls)
globe.loaded()

True

In [24]:
globe.inspect()

(0, 1, 2)

In [25]:
picks = []
picks.append(globe.pick())
picks.append(globe.pick())
picks.append(globe.pick())

In [26]:
globe.loaded()

False

In [27]:
sorted(picks) == balls

True

In [28]:
globe.load(balls)
globe.loaded()

True

In [29]:
picks = [globe.pick() for i in balls]
globe.loaded()

False

In [30]:
globe = BingoCage([])
try:
    globe.pick()
except LookupError as exc:
    print('OK')

OK


In [31]:
balls = list(range(100))
globe = BingoCage(balls)
picks = []
while globe.inspect():
    picks.append(globe.pick())
len(picks) == len(balls)

True

In [32]:
set(picks) == set(balls)

True

In [33]:
picks != balls

True

In [34]:
picks[::1] != balls

True